In [1]:
from urllib.request import urlopen, Request
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from urllib.request import urlopen
from matplotlib import pyplot as plt
import math
import datetime
from time import sleep
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr   
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from statsmodels.stats.stattools import durbin_watson

In [209]:
def AutocorrelationTest(array):
    return durbin_watson(array)

def ReduceDates(dates):
    count = 0
    for date in dates:
        if count%5 != 0:
            dates[count] = ' '
        count += 1
    return dates        

def PlotColumn(df, columns, scaler = StandardScaler(), title=None, ylabel = None, xlabel = None):
    
    #color=np.random.rand(3,), label=column
    
    dff = df[columns].copy()
    dff['date'] = df['date']
    ax = plt.gca()
    ax.locator_params(nbins=40, axis='x')
    
    for column in columns:
        data = df[[column]]
        if scaler is not None:
            dff[column] = scaler.fit_transform(df[[column]])
       
        dff.plot(kind='line',x='date',y=column,ax=ax, figsize=(20,10),label=column)
        
        plt.xticks(rotation='80')
        plt.ylabel(ylabel, fontsize=25)
        plt.xlabel(xlabel, fontsize=25)
        plt.legend()
        plt.title(title, fontsize=40)
    
    del dff['date']
    return dff

def PlotCorrelationMatrix(dff, fontsize = 20):
    f = plt.figure(figsize=(20, 10))
    data = dff.corr();
    plt.matshow(data, fignum=f.number)
    
    for (i, j), z in np.ndenumerate(data):
        plt.text(j, i, '{:0.2f}'.format(z), ha='center', va='center', fontsize=fontsize, bbox=dict(facecolor='white', alpha=0.8))
    
    plt.xticks(range(dff.shape[1]), dff.columns, fontsize=14, rotation=90)
    plt.yticks(range(dff.shape[1]), dff.columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    #plt.title('Correlation Matrix', fontsize=16);
    
def Predict(df, scaler = None, classifier = RandomForestClassifier(n_estimators=101, random_state=0), columns = None):
    print(df.shape)
    df.drop(df.tail(7).index,inplace=True) #missing data from news
    
    if columns is None:
        df = df.drop(['date','target_price', 'd_target_price', 'fear_factor_class'], axis = 1) # delete non-numeric columns
    else:
        target = df['target_rise_fall']
        df = df[columns].copy()
        df['target_rise_fall'] = target
        
    X = df.loc[:, df.columns != 'target_rise_fall']
    y = df.loc[:, df.columns == 'target_rise_fall']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    X_train_r = X_train.copy()
    if scaler is not None:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    classifier.fit(X_train, y_train.values.ravel())
    y_pred = classifier.predict(X_test)
    print("---------------------Summary------------------")
    print(classification_report(y_test,y_pred))
    print("----------------Confusion_matrix--------------")
    print(confusion_matrix(y_test,y_pred))
    print("--------------------Accuracy------------------")
    print(accuracy_score(y_test, y_pred))
    return  classifier , X_train_r, accuracy_score(y_test, y_pred)
    
def DefineThreeClasses(df):
    df['target_rise_fall'] = np.where((df.d_target_price > 100.00),'Rise',df.target_rise_fall)
    df['target_rise_fall'] = np.where((df.d_target_price <-100.00),'Fall',df.target_rise_fall)
    df['target_rise_fall'] = np.where(((df.d_target_price >=-100) & (df.d_target_price <=100)),'Stay',df.target_rise_fall)
    print('Rise'+ str(df[df.target_rise_fall == 'Rise'].shape[0]))
    print('Fall'+ str(df[df.target_rise_fall == 'Fall'].shape[0]))
    print('Stay'+ str(df[df.target_rise_fall == 'Stay'].shape[0]))
    return df
    

# Všetky features 

In [ ]:
dfc = PlotColumn(dff, ['target_price','d_target_price','open_price','my_feature2', 'close_price','tweet_volume', 'nonzero_positive_tweet', 'sum_compound', 'nonzero_compound_tweet', 'total_trans_fees', 'miners_revenue', 'market_cap','market_volume', 'num_of_transaction', 'bitcoin_trends', 'cobalt_price', 'oil_price', 'amd_price', 'intel_price', 'nvidia_price'], scaler = StandardScaler(), title ='Porovnanie objemu tweetov a objemu zložiek sentimentu\n a ich diferencie',  ylabel = 'Cena', xlabel = 'Dátum')
PlotCorrelationMatrix(dfc, fontsize = 8)
print("<0,2) - positive correlation ; (2,4> - negative correlation; 2 - no correlation")

# SUMAR TWITTER DATA - úroveň jednotlivých častí sentimentu bez a so zohladnením počtu followerov


https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC  
 - SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True, probability=False, tol=0.001,    cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)  
 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier  
 - RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
 
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB
 - GaussianNB(priors=None, var_smoothing=1e-09)
 
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier  
 - KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)


In [217]:
def PrepareDataset():
    df = pd.read_csv("data_d.csv")
    dff = df
    dff['target_price'] = dff['close_price'].shift(-1) 
    dff['d_target_price'] = np.append(np.diff(dff['close_price']), 0)
    #dff['d_price_move'] = dff.close_price - dff.open_price
    dff['d_tweet_volume'] = np.insert(np.diff(dff['tweet_volume']), 0, 0)  
    dff['my_feature2'] = dff.tweet_volume * (df.avg_sum_positive-df.avg_sum_negative)
    
    dff = dff[:-1]
    dff = dff.iloc[1:]
    dff['target_rise_fall'] = (dff.d_target_price>0.0)
    return dff


# Random Forrest whole set
Results for 100 runs:
3classes - 48%
2classes - 51%

In [260]:
###CLASSIFICATION###DONT MOVE PrepareDataset()  TO SEPARATE BLOCK  - DF must be initialized every time
suma = 0
for x in range(100):
    dff = PrepareDataset()
    dff= DefineThreeClasses(dff)
    dff = shuffle(dff)
    clf, X_train, acc = Predict(dff,scaler = StandardScaler(), classifier = RandomForestClassifier(n_estimators=101, max_depth = 10,min_samples_split=8))
    suma += acc
    
    
feature_importances = pd.DataFrame(clf.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances.head(10)
print("AVG "+str(suma/100))

Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.67      0.15      0.25        13
        Rise       0.43      1.00      0.60        13
        Stay       0.93      0.64      0.76        22

    accuracy                           0.60        48
   macro avg       0.68      0.60      0.54        48
weighted avg       0.73      0.60      0.58        48

----------------Confusion_matrix--------------
[[ 2 10  1]
 [ 0 13  0]
 [ 1  7 14]]
--------------------Accuracy------------------
0.6041666666666666
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.44      0.62      0.52        13
        Rise       0.40      0.32      0.35        19
        Stay       0.60      0.56      0.58        16

    accuracy                           0.48        48
   macro avg       0.48      0.50      

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.20      0.09      0.13        11
        Rise       0.48      0.62      0.54        21
        Stay       0.56      0.56      0.56        16

    accuracy                           0.48        48
   macro avg       0.41      0.42      0.41        48
weighted avg       0.44      0.48      0.45        48

----------------Confusion_matrix--------------
[[ 1  9  1]
 [ 2 13  6]
 [ 2  5  9]]
--------------------Accuracy------------------
0.4791666666666667
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.33      0.17      0.22        12
        Rise       0.37      0.59      0.45        17
        Stay       0.60      0.47      0.53        19

    accuracy                           0.44        48
   macro avg       0.43      0.41      0.40        48
weighted avg     

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.67      0.11      0.19        18
        Rise       0.39      0.60      0.47        15
        Stay       0.41      0.60      0.49        15

    accuracy                           0.42        48
   macro avg       0.49      0.44      0.38        48
weighted avg       0.50      0.42      0.37        48

----------------Confusion_matrix--------------
[[2 8 8]
 [1 9 5]
 [0 6 9]]
--------------------Accuracy------------------
0.4166666666666667
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.43      0.20      0.27        15
        Rise       0.42      0.50      0.46        16
        Stay       0.59      0.76      0.67        17

    accuracy                           0.50        48
   macro avg       0.48      0.49      0.47        48
weighted avg       0.48   

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.38      0.16      0.22        19
        Rise       0.33      0.35      0.34        17
        Stay       0.32      0.58      0.41        12

    accuracy                           0.33        48
   macro avg       0.34      0.36      0.33        48
weighted avg       0.35      0.33      0.31        48

----------------Confusion_matrix--------------
[[3 8 8]
 [4 6 7]
 [1 4 7]]
--------------------Accuracy------------------
0.3333333333333333
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.67      0.22      0.33        18
        Rise       0.32      0.55      0.40        11
        Stay       0.61      0.74      0.67        19

    accuracy                           0.50        48
   macro avg       0.53      0.50      0.47        48
weighted avg       0.56   

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.00      0.00      0.00         8
        Rise       0.44      0.50      0.47        16
        Stay       0.79      0.79      0.79        24

    accuracy                           0.56        48
   macro avg       0.41      0.43      0.42        48
weighted avg       0.54      0.56      0.55        48

----------------Confusion_matrix--------------
[[ 0  7  1]
 [ 4  8  4]
 [ 2  3 19]]
--------------------Accuracy------------------
0.5625
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.60      0.30      0.40        10
        Rise       0.62      0.40      0.49        25
        Stay       0.33      0.69      0.45        13

    accuracy                           0.46        48
   macro avg       0.52      0.46      0.45        48
weighted avg       0.54      

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       1.00      0.27      0.42        15
        Rise       0.47      0.47      0.47        17
        Stay       0.48      0.81      0.60        16

    accuracy                           0.52        48
   macro avg       0.65      0.52      0.50        48
weighted avg       0.64      0.52      0.50        48

----------------Confusion_matrix--------------
[[ 4  6  5]
 [ 0  8  9]
 [ 0  3 13]]
--------------------Accuracy------------------
0.5208333333333334
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.17      0.09      0.12        11
        Rise       0.41      0.41      0.41        22
        Stay       0.40      0.53      0.46        15

    accuracy                           0.38        48
   macro avg       0.33      0.34      0.33        48
weighted avg     

C:\Users\kriza\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.50      0.23      0.32        13
        Rise       0.55      0.67      0.60        18
        Stay       0.60      0.71      0.65        17

    accuracy                           0.56        48
   macro avg       0.55      0.53      0.52        48
weighted avg       0.55      0.56      0.54        48

----------------Confusion_matrix--------------
[[ 3  6  4]
 [ 2 12  4]
 [ 1  4 12]]
--------------------Accuracy------------------
0.5625
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.29      0.40      0.33        10
        Rise       0.47      0.33      0.39        21
        Stay       0.47      0.53      0.50        17

    accuracy                           0.42        48
   macro avg       0.41      0.42      0.41        48
weighted avg       0.43      

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.33      0.13      0.19        15
        Rise       0.33      0.57      0.42        14
        Stay       0.61      0.58      0.59        19

    accuracy                           0.44        48
   macro avg       0.43      0.43      0.40        48
weighted avg       0.44      0.44      0.42        48

----------------Confusion_matrix--------------
[[ 2 10  3]
 [ 2  8  4]
 [ 2  6 11]]
--------------------Accuracy------------------
0.4375
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.36      0.45      0.40        11
        Rise       0.31      0.29      0.30        17
        Stay       0.61      0.55      0.58        20

    accuracy                           0.44        48
   macro avg       0.43      0.43      0.43        48
weighted avg       0.45      

# GaussianNB - whole dataset
Results for 100 runs:
3classes - 44%
2classes - 54%

In [229]:
###CLASSIFICATION###DONT MOVE THIS SECTION TO SEPARATE BLOCK
suma = 0
for x in range(100):
    dff = PrepareDataset()
    #dff= DefineThreeClasses(dff)
    dff = shuffle(dff)
    clf, X_train, acc = Predict(dff,scaler = StandardScaler() , classifier = GaussianNB())
    suma += acc

print("AVG "+str(suma/100))

(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.56      0.40      0.47        25
        True       0.50      0.65      0.57        23

    accuracy                           0.52        48
   macro avg       0.53      0.53      0.52        48
weighted avg       0.53      0.52      0.51        48

----------------Confusion_matrix--------------
[[10 15]
 [ 8 15]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.33      0.33      0.33        18
        True       0.60      0.60      0.60        30

    accuracy                           0.50        48
   macro avg       0.47      0.47      0.47        48
weighted avg       0.50      0.50      0.50        48

----------------Confusion_matrix--------------
[[ 6 12]
 [12 18]]
--------------------Accuracy----

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.56      0.21      0.30        24
        True       0.51      0.83      0.63        24

    accuracy                           0.52        48
   macro avg       0.53      0.52      0.47        48
weighted avg       0.53      0.52      0.47        48

----------------Confusion_matrix--------------
[[ 5 19]
 [ 4 20]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.62      0.38      0.47        21
        True       0.63      0.81      0.71        27

    accuracy                           0.62        48
   macro avg       0.62      0.60      0.59        48
weighted avg       0.62      0.62      0.61        48

----------------Confusion_matrix--------------
[[ 8 13]
 [ 5 22]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.44      0.19      0.27        21
        True       0.56      0.81      0.67        27

    accuracy                           0.54        48
   macro avg       0.50      0.50      0.47        48
weighted avg       0.51      0.54      0.49        48

----------------Confusion_matrix--------------
[[ 4 17]
 [ 5 22]]
--------------------Accuracy------------------
0.5416666666666666
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.45      0.41      0.43        22
        True       0.54      0.58      0.56        26

    accuracy                           0.50        48
   macro avg       0.49      0.49      0.49        48
weighted avg       0.50      0.50      0.50        48

----------------Confusion_matrix--------------
[[ 9 13]
 [11 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.38      0.25      0.30        20
        True       0.57      0.71      0.63        28

    accuracy                           0.52        48
   macro avg       0.48      0.48      0.47        48
weighted avg       0.49      0.52      0.50        48

----------------Confusion_matrix--------------
[[ 5 15]
 [ 8 20]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.36      0.20      0.26        20
        True       0.57      0.75      0.65        28

    accuracy                           0.52        48
   macro avg       0.47      0.47      0.45        48
weighted avg       0.48      0.52      0.48        48

----------------Confusion_matrix--------------
[[ 4 16]
 [ 7 21]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.60      0.36      0.45        25
        True       0.52      0.74      0.61        23

    accuracy                           0.54        48
   macro avg       0.56      0.55      0.53        48
weighted avg       0.56      0.54      0.53        48

----------------Confusion_matrix--------------
[[ 9 16]
 [ 6 17]]
--------------------Accuracy------------------
0.5416666666666666
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.75      0.30      0.43        30
        True       0.42      0.83      0.56        18

    accuracy                           0.50        48
   macro avg       0.58      0.57      0.49        48
weighted avg       0.62      0.50      0.48        48

----------------Confusion_matrix--------------
[[ 9 21]
 [ 3 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.75      0.33      0.46        27
        True       0.50      0.86      0.63        21

    accuracy                           0.56        48
   macro avg       0.62      0.60      0.55        48
weighted avg       0.64      0.56      0.54        48

----------------Confusion_matrix--------------
[[ 9 18]
 [ 3 18]]
--------------------Accuracy------------------
0.5625
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.35      0.38      0.36        21
        True       0.48      0.44      0.46        27

    accuracy                           0.42        48
   macro avg       0.41      0.41      0.41        48
weighted avg       0.42      0.42      0.42        48

----------------Confusion_matrix--------------
[[ 8 13]
 [15 12]]
--------------------Accuracy------------------
0.416666

# SVM - whole dataset
Results for 100 runs:
3classes - 42%
2classes - 58%

In [ ]:
###CLASSIFICATION###DONT MOVE THIS SECTION TO SEPARATE BLOCK |
suma = 0
for x in range(100):
    dff = PrepareDataset()
    dff= DefineThreeClasses(dff)
    dff = shuffle(dff)
    clf, X_train, acc = Predict(dff,scaler = StandardScaler(), classifier = svm.LinearSVC(penalty='l1',loss='squared_hinge',dual=False, max_iter = 15000))
    #pd.Series(abs(clf.coef_[0]), index=X_train.columns).nlargest(20).plot(kind='barh')
    suma += acc
    
print("AVG "+str(suma/100))

Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.38      0.55      0.44        11
        Rise       0.56      0.39      0.46        23
        Stay       0.38      0.43      0.40        14

    accuracy                           0.44        48
   macro avg       0.44      0.46      0.44        48
weighted avg       0.46      0.44      0.44        48

----------------Confusion_matrix--------------
[[6 2 3]
 [7 9 7]
 [3 5 6]]
--------------------Accuracy------------------
0.4375
Rise73
Fall54
Stay70
(197, 105)


C:\Users\kriza\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.83      0.33      0.48        15
        Rise       0.50      0.67      0.57        18
        Stay       0.50      0.60      0.55        15

    accuracy                           0.54        48
   macro avg       0.61      0.53      0.53        48
weighted avg       0.60      0.54      0.53        48

----------------Confusion_matrix--------------
[[ 5  7  3]
 [ 0 12  6]
 [ 1  5  9]]
--------------------Accuracy------------------
0.5416666666666666
Rise73
Fall54
Stay70
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.29      0.15      0.20        13
        Rise       0.50      0.42      0.46        19
        Stay       0.36      0.56      0.44        16

    accuracy                           0.40        48
   macro avg       0.38      0.38      0.37        48
weighted avg     

C:\Users\kriza\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.31      0.25      0.28        16
        Rise       0.17      0.30      0.21        10
        Stay       0.59      0.45      0.51        22

    accuracy                           0.35        48
   macro avg       0.35      0.33      0.33        48
weighted avg       0.41      0.35      0.37        48

----------------Confusion_matrix--------------
[[ 4  9  3]
 [ 3  3  4]
 [ 6  6 10]]
--------------------Accuracy------------------
0.3541666666666667
Rise73
Fall54
Stay70
(197, 105)


...

---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.25      0.40      0.31        10
        Rise       0.50      0.32      0.39        19
        Stay       0.45      0.47      0.46        19

    accuracy                           0.40        48
   macro avg       0.40      0.40      0.39        48
weighted avg       0.43      0.40      0.40        48

----------------Confusion_matrix--------------
[[4 1 5]
 [7 6 6]
 [5 5 9]]
--------------------Accuracy------------------
0.3958333333333333
Rise73
Fall54
Stay70
(197, 105)


C:\Users\kriza\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.58      0.44      0.50        16
        Rise       0.48      0.79      0.59        14
        Stay       0.46      0.33      0.39        18

    accuracy                           0.50        48
   macro avg       0.51      0.52      0.49        48
weighted avg       0.51      0.50      0.49        48

----------------Confusion_matrix--------------
[[ 7  5  4]
 [ 0 11  3]
 [ 5  7  6]]
--------------------Accuracy------------------
0.5
Rise73
Fall54
Stay70
(197, 105)


C:\Users\kriza\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---------------------Summary------------------
              precision    recall  f1-score   support

        Fall       0.27      0.31      0.29        13
        Rise       0.50      0.32      0.39        22
        Stay       0.26      0.38      0.31        13

    accuracy                           0.33        48
   macro avg       0.34      0.34      0.33        48
weighted avg       0.37      0.33      0.34        48

----------------Confusion_matrix--------------
[[ 4  5  4]
 [ 5  7 10]
 [ 6  2  5]]
--------------------Accuracy------------------
0.3333333333333333
Rise73
Fall54
Stay70
(197, 105)


# SVC
Results for 100 runs:
3classes - 46%
2classes - 53%

In [238]:
###CLASSIFICATION###DONT MOVE THIS SECTION TO SEPARATE BLOCK |
suma = 0
for x in range(100):
    dff = PrepareDataset()
    #dff= DefineThreeClasses(dff)
    dff = shuffle(dff)
    clf, X_train, acc = Predict(dff,scaler = StandardScaler(), classifier = svm.SVC(gamma='scale'))
    suma += acc
    
print("AVG "+str(suma/100))

(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.50      0.50      0.50        20
        True       0.64      0.64      0.64        28

    accuracy                           0.58        48
   macro avg       0.57      0.57      0.57        48
weighted avg       0.58      0.58      0.58        48

----------------Confusion_matrix--------------
[[10 10]
 [10 18]]
--------------------Accuracy------------------
0.5833333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.42      0.55      0.48        20
        True       0.59      0.46      0.52        28

    accuracy                           0.50        48
   macro avg       0.51      0.51      0.50        48
weighted avg       0.52      0.50      0.50        48

----------------Confusion_matrix--------------
[[11  9]
 [15 13]]
--------------------Accuracy----

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.42      0.48      0.44        21
        True       0.54      0.48      0.51        27

    accuracy                           0.48        48
   macro avg       0.48      0.48      0.48        48
weighted avg       0.49      0.48      0.48        48

----------------Confusion_matrix--------------
[[10 11]
 [14 13]]
--------------------Accuracy------------------
0.4791666666666667
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.48      0.50      0.49        20
        True       0.63      0.61      0.62        28

    accuracy                           0.56        48
   macro avg       0.55      0.55      0.55        48
weighted avg       0.57      0.56      0.56        48

----------------Confusion_matrix--------------
[[10 10]
 [11 17]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.57      0.17      0.26        24
        True       0.51      0.88      0.65        24

    accuracy                           0.52        48
   macro avg       0.54      0.52      0.45        48
weighted avg       0.54      0.52      0.45        48

----------------Confusion_matrix--------------
[[ 4 20]
 [ 3 21]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.56      0.35      0.43        26
        True       0.47      0.68      0.56        22

    accuracy                           0.50        48
   macro avg       0.52      0.51      0.49        48
weighted avg       0.52      0.50      0.49        48

----------------Confusion_matrix--------------
[[ 9 17]
 [ 7 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.55      0.60      0.57        20
        True       0.69      0.64      0.67        28

    accuracy                           0.62        48
   macro avg       0.62      0.62      0.62        48
weighted avg       0.63      0.62      0.63        48

----------------Confusion_matrix--------------
[[12  8]
 [10 18]]
--------------------Accuracy------------------
0.625
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.80      0.32      0.46        25
        True       0.55      0.91      0.69        23

    accuracy                           0.60        48
   macro avg       0.68      0.62      0.57        48
weighted avg       0.68      0.60      0.57        48

----------------Confusion_matrix--------------
[[ 8 17]
 [ 2 21]]
--------------------Accuracy------------------
0.6041666

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.58      0.32      0.41        22
        True       0.58      0.81      0.68        26

    accuracy                           0.58        48
   macro avg       0.58      0.56      0.54        48
weighted avg       0.58      0.58      0.56        48

----------------Confusion_matrix--------------
[[ 7 15]
 [ 5 21]]
--------------------Accuracy------------------
0.5833333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.41      0.37      0.39        19
        True       0.61      0.66      0.63        29

    accuracy                           0.54        48
   macro avg       0.51      0.51      0.51        48
weighted avg       0.53      0.54      0.54        48

----------------Confusion_matrix--------------
[[ 7 12]
 [10 19]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.45      0.48      0.47        21
        True       0.58      0.56      0.57        27

    accuracy                           0.52        48
   macro avg       0.52      0.52      0.52        48
weighted avg       0.52      0.52      0.52        48

----------------Confusion_matrix--------------
[[10 11]
 [12 15]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.61      0.46      0.52        24
        True       0.57      0.71      0.63        24

    accuracy                           0.58        48
   macro avg       0.59      0.58      0.58        48
weighted avg       0.59      0.58      0.58        48

----------------Confusion_matrix--------------
[[11 13]
 [ 7 17]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.54      0.70      0.61        20
        True       0.73      0.57      0.64        28

    accuracy                           0.62        48
   macro avg       0.63      0.64      0.62        48
weighted avg       0.65      0.62      0.63        48

----------------Confusion_matrix--------------
[[14  6]
 [12 16]]
--------------------Accuracy------------------
0.625
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.42      0.45      0.43        22
        True       0.50      0.46      0.48        26

    accuracy                           0.46        48
   macro avg       0.46      0.46      0.46        48
weighted avg       0.46      0.46      0.46        48

----------------Confusion_matrix--------------
[[10 12]
 [14 12]]
--------------------Accuracy------------------
0.4583333


# KNeighborsClassifier - whole dataset
Results for 100 runs:
3classes - 41%
2classes - 50%

In [240]:
suma = 0
for x in range(100):
    dff = PrepareDataset()
    #dff= DefineThreeClasses(dff)
    dff = shuffle(dff)
    clf, X_train, acc = Predict(dff,scaler = StandardScaler() , classifier = KNeighborsClassifier(n_neighbors=5))
    suma += acc
    
print("AVG "+str(suma/100))

(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.41      0.38      0.39        24
        True       0.42      0.46      0.44        24

    accuracy                           0.42        48
   macro avg       0.42      0.42      0.42        48
weighted avg       0.42      0.42      0.42        48

----------------Confusion_matrix--------------
[[ 9 15]
 [13 11]]
--------------------Accuracy------------------
0.4166666666666667
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.55      0.23      0.32        26
        True       0.46      0.77      0.58        22

    accuracy                           0.48        48
   macro avg       0.50      0.50      0.45        48
weighted avg       0.51      0.48      0.44        48

----------------Confusion_matrix--------------
[[ 6 20]
 [ 5 17]]
--------------------Accuracy----

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.38      0.22      0.28        23
        True       0.49      0.68      0.57        25

    accuracy                           0.46        48
   macro avg       0.44      0.45      0.42        48
weighted avg       0.44      0.46      0.43        48

----------------Confusion_matrix--------------
[[ 5 18]
 [ 8 17]]
--------------------Accuracy------------------
0.4583333333333333
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.27      0.35      0.31        17
        True       0.58      0.48      0.53        31

    accuracy                           0.44        48
   macro avg       0.42      0.42      0.42        48
weighted avg       0.47      0.44      0.45        48

----------------Confusion_matrix--------------
[[ 6 11]
 [16 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.42      0.36      0.39        22
        True       0.52      0.58      0.55        26

    accuracy                           0.48        48
   macro avg       0.47      0.47      0.47        48
weighted avg       0.47      0.48      0.47        48

----------------Confusion_matrix--------------
[[ 8 14]
 [11 15]]
--------------------Accuracy------------------
0.4791666666666667
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.44      0.29      0.35        24
        True       0.47      0.62      0.54        24

    accuracy                           0.46        48
   macro avg       0.45      0.46      0.44        48
weighted avg       0.45      0.46      0.44        48

----------------Confusion_matrix--------------
[[ 7 17]
 [ 9 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.46      0.30      0.36        20
        True       0.60      0.75      0.67        28

    accuracy                           0.56        48
   macro avg       0.53      0.53      0.52        48
weighted avg       0.54      0.56      0.54        48

----------------Confusion_matrix--------------
[[ 6 14]
 [ 7 21]]
--------------------Accuracy------------------
0.5625
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.38      0.38      0.38        21
        True       0.52      0.52      0.52        27

    accuracy                           0.46        48
   macro avg       0.45      0.45      0.45        48
weighted avg       0.46      0.46      0.46        48

----------------Confusion_matrix--------------
[[ 8 13]
 [13 14]]
--------------------Accuracy------------------
0.458333

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.37      0.33      0.35        21
        True       0.52      0.56      0.54        27

    accuracy                           0.46        48
   macro avg       0.44      0.44      0.44        48
weighted avg       0.45      0.46      0.45        48

----------------Confusion_matrix--------------
[[ 7 14]
 [12 15]]
--------------------Accuracy------------------
0.4583333333333333
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.40      0.44      0.42        18
        True       0.64      0.60      0.62        30

    accuracy                           0.54        48
   macro avg       0.52      0.52      0.52        48
weighted avg       0.55      0.54      0.55        48

----------------Confusion_matrix--------------
[[ 8 10]
 [12 18]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.53      0.38      0.44        24
        True       0.52      0.67      0.58        24

    accuracy                           0.52        48
   macro avg       0.52      0.52      0.51        48
weighted avg       0.52      0.52      0.51        48

----------------Confusion_matrix--------------
[[ 9 15]
 [ 8 16]]
--------------------Accuracy------------------
0.5208333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.45      0.41      0.43        22
        True       0.54      0.58      0.56        26

    accuracy                           0.50        48
   macro avg       0.49      0.49      0.49        48
weighted avg       0.50      0.50      0.50        48

----------------Confusion_matrix--------------
[[ 9 13]
 [11 15]]
--------------------Accuracy---------------

---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.52      0.57      0.55        21
        True       0.64      0.59      0.62        27

    accuracy                           0.58        48
   macro avg       0.58      0.58      0.58        48
weighted avg       0.59      0.58      0.58        48

----------------Confusion_matrix--------------
[[12  9]
 [11 16]]
--------------------Accuracy------------------
0.5833333333333334
(197, 105)
---------------------Summary------------------
              precision    recall  f1-score   support

       False       0.16      0.16      0.16        19
        True       0.45      0.45      0.45        29

    accuracy                           0.33        48
   macro avg       0.30      0.30      0.30        48
weighted avg       0.33      0.33      0.33        48

----------------Confusion_matrix--------------
[[ 3 16]
 [16 13]]
--------------------Accuracy---------------

# Try for Votting classifiers

In [ ]:
def Voting(df, columns = None, scaler = StandardScaler()):
    print(df.shape)
    df.drop(df.tail(7).index,inplace=True) #missing data from news
    
    if columns is None:
        df = df.drop(['date','target_price', 'd_target_price'], axis = 1) # delete non-numeric columns
    else:
        target = df['target_rise_fall']
        df = df[columns].copy()
        df['target_rise_fall'] = target
        
    X = df.loc[:, df.columns != 'target_rise_fall']
    y = df.loc[:, df.columns == 'target_rise_fall']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    X_train_r = X_train.copy()
    if scaler is not None:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    gnb = GaussianNB()
    rf = RandomForestClassifier(n_estimators=101)
    svml = svm.LinearSVC()
    svc = svm.SVC(gamma='scale')
    knb = KNeighborsClassifier(n_neighbors=5)
    
    gnb.fit(X_train, y_train.values.ravel())
    gnb_predict = gnb.predict(X_test)
    
    rf.fit(X_train, y_train.values.ravel())
    rf_predict = rf.predict(X_test)
    
    svml.fit(X_train, y_train.values.ravel())
    svm_predict = svml.predict(X_test)
    
    svc.fit(X_train, y_train.values.ravel())
    svc_predict = svc.predict(X_test)
    
    knb.fit(X_train, y_train.values.ravel())
    knb_predict = knb.predict(X_test)
    
    res = pd.DataFrame(gnb_predict, columns = ['gnb_predict']) 
    res['rf_predict'] = rf_predict
    res['svm_predict'] = svm_predict
    res['svc_predict'] = svc_predict
    res['knb_predict'] = knb_predict
    
    res['tftarget'] = np.array(y_test)
    return res

In [ ]:
dff = PrepareDataset()
res = Voting(dff, columns = GetColumnSet('myfeatures'), scaler = StandardScaler())

In [ ]:
res.head(5)

In [ ]:
for index, row in res.iterrows():
    count = 0
    if row['gnb_predict'] == True:
        count +=1
    if row['rf_predict'] == True:
        count +=1
    if row['svm_predict'] == True:
        count +=1
    if row['svc_predict'] == True:
        count +=1
    if row['knb_predict'] == True:
        count +=1
    if count >=3:
        res.set_value(index, 'vote_res', True)
        count = 0
        continue
    res.set_value(index, 'vote_res', False)
    count = 0
    

In [ ]:
res.head(50)

In [ ]:

print("---------------------Summary------------------")
print(classification_report(res.tftarget.values.tolist(), res.vote_res.values.tolist()))
print("----------------Confusion_matrix--------------")
print(confusion_matrix(res.tftarget.values.tolist(), res.vote_res.values.tolist()))
print("--------------------Accuracy------------------")
print(accuracy_score(res.tftarget.values.tolist(), res.vote_res.values.tolist()))